In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


In [5]:
sc.textFile("/labs/laba01/ml-100k/README").take(10)

['SUMMARY & USAGE LICENSE',
 '=============================================',
 '',
 'MovieLens data sets were collected by the GroupLens Research Project',
 'at the University of Minnesota.',
 ' ',
 'This data set consists of:',
 '\t* 100,000 ratings (1-5) from 943 users on 1682 movies. ',
 '\t* Each user has rated at least 20 movies. ',
 '        * Simple demographic info for the users (age, gender, occupation, zip)']

**Load Data**

In [6]:
data = sc.textFile("/labs/laba01/ml-100k/u.data")
item = sc.textFile("/labs/laba01/ml-100k/u.item")

In [7]:
data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [8]:
print('Num partitions: ', data.getNumPartitions())
print('Lines: ', data.count())

Num partitions:  2
Lines:  100000


In [9]:
data_sep = data.map(lambda x: x.split("\t"))

In [10]:
data_sep.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [11]:
# users
data_sep.map(lambda x: x[0]).distinct().count()

943

In [12]:
# films
data_sep.map(lambda x: x[1]).distinct().count()

1682

In [13]:
# ratings
data_sep.map(lambda x: x[2]).distinct().count()

5

In [14]:
# ratings
data_sep.map(lambda x: x[3]).distinct().count()

49282

**Get value counts**

In [15]:
selected_film = data_sep.filter(lambda x: x[1] == "328")
sorted(selected_film.map(lambda x: x[2]).countByValue().items())

[('1', 12), ('2', 40), ('3', 94), ('4', 109), ('5', 40)]

In [16]:
sorted(data_sep.map(lambda x: x[2]).countByValue().items())

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

**Create json file**

In [27]:
hist_film_lst = sorted(selected_film.map(lambda x: x[2]).countByValue().items())
hist_all_lst = sorted(data_sep.map(lambda x: x[2]).countByValue().items())

In [30]:
hist_film = []
for i in range(len(hist_film_lst)):
    hist_film.append(hist_film_lst[i][1])
hist_film

[12, 40, 94, 109, 40]

In [28]:
hist_all = []
for i in range(len(el)):
    hist_all.append(el[i][1])
hist_all

[6110, 11370, 27145, 34174, 21201]

In [32]:
json_file = {
   "hist_film": hist_film,
   "hist_all": hist_all
}

In [33]:
json_file

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [97]:
import json

with open('lab01.json', 'w') as fp:
    json.dump(json_file, fp)

**Put into server**

In [19]:
! hdfs dfs -ls /user/dmitry.ulogov

Found 1 items
drwxr-xr-x   - dmitry.ulogov dmitry.ulogov          0 2021-02-26 21:39 /user/dmitry.ulogov/.sparkStaging


In [20]:
! hdfs dfs -put lab01.json /user/dmitry.ulogov

In [21]:
! hdfs dfs -ls /user/dmitry.ulogov

Found 2 items
drwxr-xr-x   - dmitry.ulogov dmitry.ulogov          0 2021-02-26 21:39 /user/dmitry.ulogov/.sparkStaging
-rw-r--r--   3 dmitry.ulogov dmitry.ulogov         84 2021-02-26 21:44 /user/dmitry.ulogov/lab01.json


In [34]:
sc.stop()